In [ ]:
import random
import tqdm

!curl -L -o train.txt https://github.com/neubig/anlp-code/raw/refs/heads/main/data/sst-sentiment-text-threeclass/train.txt
!curl -L -o dev.txt https://github.com/neubig/anlp-code/raw/refs/heads/main/data/sst-sentiment-text-threeclass/dev.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100  918k  100  918k    0     0  4270k      0 --:--:-- --:--:-- --:--:-- 4270k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100  119k  100  119k    0     0   126k      0 --:--:-- --:--:-- --:--:--  126k


In [23]:
def extract_features(x: str) -> dict[str, float]:
    features = {}
    x_split = x.split(' ')
    for x in x_split:
        features[x] = features.get(x, 0) + 1.0
    return features

In [24]:
feature_weights = {}

In [29]:
def read_xy_data(filename: str) -> tuple[list[str], list[int]]:
    x_data = []
    y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            x_data.append(text)
            y_data.append(int(label))
    return x_data, y_data

In [30]:
x_train, y_train = read_xy_data('train.txt')
x_dev, y_dev = read_xy_data('dev.txt')

In [33]:
print(x_train[0])
print(y_train[0])

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
1


In [34]:
def run_classifier(features: dict[str, float]) -> int:
    score = 0
    for feat_name, feat_value in features.items():
        score = score + feat_value * feature_weights.get(feat_name, 0)
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

In [35]:
NUM_EPOCHS = 5
for epoch in range(1, NUM_EPOCHS+1):
    # Shuffle the order of the data
    data_ids = list(range(len(x_train)))
    random.shuffle(data_ids)
    # Run over all data points
    for data_id in tqdm.tqdm(data_ids, desc=f'Epoch {epoch}'):
        x = x_train[data_id]
        y = y_train[data_id]
        # We will skip neutral examples
        if y == 0:
            continue
        # Make a prediction
        features = extract_features(x)
        predicted_y = run_classifier(features)
        # Update the weights if the prediction is wrong
        if predicted_y != y:
            for feature in features:
                feature_weights[feature] = feature_weights.get(feature, 0) + y * features[feature]

Epoch 5: 100%|██████████| 8544/8544 [00:00<00:00, 146567.85it/s]


In [36]:
print(f"Total features {len(feature_weights.keys())}")
for i,(k,v) in enumerate(feature_weights.items()):
  print(f"{k}\t\t{v}")
  if i>20:
    break

Total features 13159
A		2.0
hypnotic		3.0
cyber		-1.0
hymn		1.0
and		4.0
a		3.0
cruel		-1.0
story		6.0
of		2.0
youth		-1.0
culture		7.0
.		0.0
What		2.0
goes		2.0
on		1.0
for		-1.0
the		-1.0
110		-1.0
minutes		-6.0
``		0.0
Panic		2.0
Room		3.0


In [37]:
def calculate_accuracy(x_data: list[str], y_data: list[int]) -> float:
    total_number = 0
    correct_number = 0
    for x, y in zip(x_data, y_data):
        y_pred = run_classifier(extract_features(x))
        total_number += 1
        if y == y_pred:
            correct_number += 1
    return correct_number / float(total_number)

In [38]:

label_count = {}
for y in y_dev:
    if y not in label_count:
        label_count[y] = 0
    label_count[y] += 1
print(label_count)
     

{1: 444, 0: 229, -1: 428}


In [39]:

train_accuracy = calculate_accuracy(x_train, y_train)
test_accuracy = calculate_accuracy(x_dev, y_dev)
print(f'Train accuracy: {train_accuracy}')
print(f'Dev/test accuracy: {test_accuracy}')

Train accuracy: 0.7439138576779026
Dev/test accuracy: 0.584014532243415
